In [51]:
import numpy as np
from scipy.stats import binom
import pandas as pd
from itertools import combinations

carga = 2850

In [52]:
class Agrupamento:
    def __init__(self, pot, qtd, prob):
        self.pot = pot
        self.qtd = qtd
        self.prob = prob
        self.geraTabela()

    def geraTabela(self):
        self.resultados = []
        self.iterado = []
        for i in range(int(self.qtd) + 1):
            self.resultados.append((i * self.pot, binom.pmf(i, int(self.qtd), self.prob)))
            self.iterado.append(0)
        self.usina = pd.DataFrame(self.resultados,columns=['Pot','Prob'])
        self.usina['Iter'] = self.iterado
        self.usina['IterAg'] = np.nan
            
            
def pu(x):
    return x/100

In [53]:
data = pd.read_excel('Gerac.xlsx')
data['FOR'] = data['FOR'].apply(pu)

In [54]:
agrupamento = []
usinas = []
geracao_total = 0
for i in range(len(data['Usina'])):
    agrupamento.append(Agrupamento(data.loc[i]['Pot. Ativa Max'], data.loc[i]['Unidades'], data.loc[i]['FOR']))
    usinas.append(agrupamento[i].usina)
    geracao_total += data.loc[i]['Unidades']*data.loc[i]['Pot. Ativa Max']

potencia_para_falta = geracao_total - carga

lista = []
for i in range(len(usinas)):
    for x in range(len(usinas[i])):
        lista.append((i,x))

combinacoes = []
for i in combinations(lista,2):
    combinacoes.append(i)

In [55]:
resultados_final = []

for i in range(len(combinacoes)):
    soma = 0
    produto = 1
    for j in combinacoes[i]:
        soma += agrupamento[j[0]].resultados[j[1]][0]
        produto *= agrupamento[j[0]].resultados[j[1]][1]
    resultados_final.append((soma,produto))

In [56]:
copt = pd.DataFrame(resultados_final, columns=['Pot indisponivel', 'Probabilidade'])
copt = copt.groupby(by = ['Pot indisponivel'], as_index = False).sum()


In [57]:
perda_de_carga = []

for i in range(len(copt)):
    if copt.loc[i]['Pot indisponivel'] <=  potencia_para_falta:
        perda_de_carga.append(0)
    else:
        perda_de_carga.append(abs(copt.loc[i]['Pot indisponivel'] - potencia_para_falta))

In [58]:
copt['Perda de Carga'] = perda_de_carga
copt['x.p'] = copt['Perda de Carga'] * copt['Probabilidade']

# mudar formato da probabilidade para % com 2 casas decimais
copt['Probabilidade'] = copt['Probabilidade'].apply(lambda x: round(x, 4))
copt

,Pot indisponivel,Probabilidade,Perda de Carga,x.p
0,0.0,74.2771,0.0,0.000000
1,12.0,1.1668,0.0,0.000000
2,20.0,4.5540,0.0,0.000000
3,24.0,0.0476,0.0,0.000000
4,32.0,0.0332,0.0,0.000000
...,...,...,...,...
203,891.0,0.0000,336.0,0.000003
204,901.0,0.0000,346.0,0.000069
205,941.0,0.0000,386.0,0.003860
206,985.0,0.0000,430.0,0.000383


In [59]:
EPNS = sum(copt['x.p'])
LOLP = sum(copt[copt['Perda de Carga'] > 0]['Probabilidade'])
LOLE = LOLP * 8760
EENS = EPNS * 8760

In [60]:
print('A LOLP do problema é: {}'.format(LOLP))
print('A EPNS do problema é: {} MW'.format(EPNS))
print('A LOLE do problema é: {} horas'.format(LOLE))
print('A EENS do problema é: {} MWh'.format(EENS))


A LOLP do problema é: 0.07250000000000001
A EPNS do problema é: 9.405282349345297 MW
A LOLE do problema é: 635.1 horas
A EENS do problema é: 82390.2733802648 MWh
